In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from global_import import *
from valuation.val import perform_valuation

In [2]:
data = pd.read_csv(f'{DATA_FOLDER}/lagged_comp-fundr-ibes_for_val_1976-2019.csv', index_col = 0)
crsp = pd.read_csv(f'{DATA_FOLDER}/crsp_1959-2019.csv', index_col=0)
beta = pd.read_csv(f'{DATA_FOLDER}/capm_beta_weekly_1-3y_1959-2019.csv', index_col=0)

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data = data.replace([np.inf, -np.inf], np.nan)
data = data.drop_duplicates(subset=['jdate', 'permno'])

In [4]:
X = data.drop(['ticker', 'exchcd', 'permno','jdate','year','ni_1', 'ni_2', 'ni_3','ni_4','ni_5'], axis = 1)

X = X.fillna(0).astype(float)

print(X.shape)

(323254, 301)


In [5]:
data[data.ticker=='TSLA']

,ticker,exchcd,permno,jdate,count,shrout,sic2,sic,siccd,naics,act,ceq,che,dltt,gdwl,intan,lct,lt,ppegt,ppent,txdb,wcap,dp,dvt,ebit,ebitda,epsfx,epspx,gp,ib,ni,niadj,nopi,opincar,opiti,pi,revt,capx,aqc,dv,chech,dpc,wcapch,ps_x,be,ni_-5,at_-5,epspx_-5,revt_-5,capx_-5,naics_-5,ib_-5,ebitda_-5,ebit_-5,nopi_-5,pi_-5,dvt_-5,be_-5,ni_-4,at_-4,epspx_-4,revt_-4,capx_-4,naics_-4,ib_-4,ebitda_-4,ebit_-4,nopi_-4,pi_-4,dvt_-4,be_-4,ni_-3,at_-3,epspx_-3,revt_-3,capx_-3,naics_-3,ib_-3,ebitda_-3,ebit_-3,nopi_-3,pi_-3,dvt_-3,be_-3,ni_-2,at_-2,epspx_-2,revt_-2,capx_-2,naics_-2,ib_-2,ebitda_-2,ebit_-2,nopi_-2,pi_-2,dvt_-2,be_-2,ni_-1,at_-1,epspx_-1,revt_-1,capx_-1,naics_-1,ib_-1,ebitda_-1,ebit_-1,nopi_-1,pi_-1,dvt_-1,be_-1,ni_0,ni_1,ni_2,ni_3,ni_4,ni_5,at_yoy1,ni_yoy1,epspx_yoy1,revt_yoy1,capx_yoy1,naics_yoy1,ib_yoy1,ebitda_yoy1,ebit_yoy1,nopi_yoy1,pi_yoy1,dvt_yoy1,be_yoy1,at_yoy2,ni_yoy2,epspx_yoy2,revt_yoy2,capx_yoy2,naics_yoy2,ib_yoy2,ebitda_yoy2,ebit_yoy2,nopi_yoy2,pi_yoy2,dvt_yoy2,be_yoy2,at_yoy3,ni_yoy3,epspx_yoy3,revt_yoy3,capx_yoy3,naics_yoy3,ib_yoy3,ebitda_yoy3,ebit_yoy3,nopi_yoy3,pi_yoy3,dvt_yoy3,be_yoy3,at_yoy4,ni_yoy4,epspx_yoy4,revt_yoy4,capx_yoy4,naics_yoy4,ib_yoy4,ebitda_yoy4,ebit_yoy4,nopi_yoy4,pi_yoy4,dvt_yoy4,be_yoy4,at_yoy5,ni_yoy5,epspx_yoy5,revt_yoy5,capx_yoy5,naics_yoy5,ib_yoy5,ebitda_yoy5,ebit_yoy5,nopi_yoy5,pi_yoy5,dvt_yoy5,be_yoy5,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,pcf,dpr,npm,opmbd,opmad,gpm,ptpm,cfm,roa,roe,roce,efftax,aftret_eq,aftret_invcapx,aftret_equity,pretret_noa,pretret_earnat,GProf,equity_invcap,debt_invcap,totdebt_invcap,capital_ratio,int_debt,int_totdebt,cash_lt,invt_act,rect_act,debt_at,debt_ebitda,short_debt,curr_debt,lt_debt,profit_lct,ocf_lct,cash_debt,fcf_ocf,lt_ppent,dltt_be,debt_assets,debt_capital,de_ratio,intcov,intcov_ratio,cash_ratio,quick_ratio,curr_ratio,cash_conversion,inv_turn,at_turn,rect_turn,pay_turn,sale_invcap,sale_equity,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,PEG_1yrforward,PEG_ltgforward,med_eps_1,mean_eps_1,std_eps_1,med_sal_1,mean_sal_1,std_sal_1,med_pre_1,mean_pre_1,std_pre_1,med_ebt_1,mean_ebt_1,std_ebt_1,med_eps_2,mean_eps_2,std_eps_2,med_sal_2,mean_sal_2,std_sal_2,med_pre_2,mean_pre_2,std_pre_2,med_ebt_2,mean_ebt_2,std_ebt_2,med_eps_3,mean_eps_3,std_eps_3,med_sal_3,mean_sal_3,std_sal_3,med_pre_3,mean_pre_3,std_pre_3,med_ebt_3,mean_ebt_3,std_ebt_3,med_eps_4,mean_eps_4,std_eps_4,med_sal_4,mean_sal_4,std_sal_4,med_pre_4,mean_pre_4,std_pre_4,med_ebt_4,mean_ebt_4,std_ebt_4,med_eps_5,mean_eps_5,std_eps_5,med_sal_5,mean_sal_5,std_sal_5,med_pre_5,mean_pre_5,std_pre_5,med_ebt_5,mean_ebt_5,std_ebt_5,year
692368,TSLA,3.0,93436,2019-06-30,10,179118.0,3711,3711,9999.0,336111.0,8306.308,4923.243,3878.169,9454.055,68.159,350.651,9992.136,23426.010,23343.447,19691.231,0.000,-1685.828,1887.786,0.0,-252.840,1634.946,-5.72,-5.72,5929.807,-976.091,-976.091,-976.091,101.299,NaN,NaN,-1004.745,21461.268,2319.516,17.912,0.0,311.429,2046.516,NaN,0.000,4923.243,-74.014,2416.930,-0.62,2013.496,264.224,336111.0,-74.014,44.800,-61.283,26.292,-71.426,0.0,673.941,-294.040,5849.251,-2.36,3198.356,969.885,336111.0,-294.040,45.242,-186.689,15.701,-284.636,0.0,969.906,-888.663,8092.460,-6.93,4046.025,1634.850,336111.0,-888.663,-294.039,-716.629,1.333,-875.624,0.0,1130.989,-674.914,22664.076,-4.68,7000.132,1440.471,336111.0,-674.914,301.459,-645.640,84.966,-746.348,0.0,4761.695,-1961.400,28655.372,-11.83,11758.751,4081.354,336111.0,-1961.400,70.917,-1565.086,76.972,-2209.032,0.0,4237.312,-976.091,-862.000,NaN,NaN,NaN,NaN,0.036458,-1.009444,-1.068182,0.452094,-0.759571,0.0,-1.009444,0.956624,-5.190025,0.240150,-1.198600,NaN,0.139325,0.487767,0.555477,0.426401,0.820868,0.615612,0.0,0.555477,0.903114,NaN,0.401541,0.507126,NaN,0.181145,0.899543,0.447425,NaN,0.932735,0.665826,0.0,0.447425,1.056676,NaN,0.995594,0.504638,NaN,0.916675,0.946721,0.914285,0.875459,0.960459,0.873382,0.0,0.914285,0.993009,0.715192,0.958770,0.920101,NaN,0.946625,0.983190,0.984353,0.977316,0.980489,0.976102,0.0,

In [6]:
import pickle 

prepath = f'{PROJECT_FOLDER}/forecast/'
with open(f"{prepath}PLS_regr1_ncomp4.pickle", "rb") as input_file:
    regr1 = pickle.load(input_file)
with open(f"{prepath}PLS_regr2_ncomp4.pickle", "rb") as input_file:
    regr2 = pickle.load(input_file)
with open(f"{prepath}PLS_regr3_ncomp4.pickle", "rb") as input_file:
    regr3 = pickle.load(input_file)
with open(f"{prepath}PLS_regr4_ncomp4.pickle", "rb") as input_file:
    regr4 = pickle.load(input_file)
with open(f"{prepath}PLS_regr5_ncomp4.pickle", "rb") as input_file:
    regr5 = pickle.load(input_file)

In [7]:
ni_1_pred = regr1.predict(X)
ni_2_pred = regr2.predict(X)
ni_3_pred = regr3.predict(X)
ni_4_pred = regr4.predict(X)
ni_5_pred = regr5.predict(X)

In [8]:
mdf_rf = data[['ticker', 'permno', 'exchcd', 'year', 'jdate',
               'be']].copy()
mdf_rf['ni_1'] = ni_1_pred
mdf_rf['ni_2'] = ni_2_pred
mdf_rf['ni_3'] = ni_3_pred
mdf_rf['ni_4'] = ni_4_pred
mdf_rf['ni_5'] = ni_5_pred
for i in range(6,11):
    mdf_rf[f'ni_{i}'] = ni_5_pred
# mdf_rf.to_csv('PLS_ni_pred_for_val.csv')

In [10]:
crsp = pd.read_csv(f'{DATA_FOLDER}/crsp_1959-2019.csv', index_col=0)
beta = pd.read_csv(f'{DATA_FOLDER}/capm_beta_weekly_1-3y_1959-2019.csv', index_col=0)

comp = mdf_rf
comp['jdate'] = pd.to_datetime(comp['jdate'])
comp.permno = comp.permno.astype(int)

beta['date']=pd.to_datetime(beta['DATE'].astype(str))
beta['jdate']=beta['date']+MonthEnd(0)
beta['permno'] = beta.index
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
FF = pd.read_csv(f'{PROJECT_FOLDER}/lookup_tables/FF3.csv')
FF['date'] = FF['Unnamed: 0']
FF['date'] = pd.to_datetime(FF['date'].astype(str), format='%Y%m%d')
FF['jdate'] = FF['date']+MonthEnd(0)
FF = FF[['jdate', 'date', 'RF']]
FF['RF'] = (1+FF['RF']/100).pow(365)-1

beta = beta.merge(FF[['jdate', 'RF']],  on=['jdate'])
beta = beta.drop_duplicates(subset=['permno', 'jdate'])

In [12]:
comp.sort_values(['permno','jdate'], ascending = [True,True], inplace=True)
crsp['logret'] = np.log(1+crsp['retadj'])
crsp['termlogret'] = crsp.groupby('permno')['logret'].transform('sum') - crsp.groupby('permno')['logret'].transform('cumsum') + np.log(1-0.35) 
crsp['termcumret'] = np.exp(crsp['termlogret'])-1
crsp1 = crsp.copy()

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [13]:
for x in ['logret_3', 'logret_6', 'logret_12', 'logret_24', 
      'logret_36', 'logret_48', 'logret_60']:
    crsp1.loc[crsp1[x].isnull(), x] = crsp1['termlogret']


for x in ['cumret_3', 'cumret_6', 'cumret_12', 'cumret_24', 
      'cumret_36', 'cumret_48', 'cumret_60']:
    crsp1.loc[crsp1[x].isnull(), x] = crsp1['termcumret']

In [14]:
valdf = comp.merge(beta, on=['jdate', 'permno'])
valdf = perform_valuation(valdf, n=5, rp=0.07, tg=0.05)
valdf = valdf[['permno', 'year', 'PV', 'be']]
valdf.year = valdf.year + 1
val_crsp_mdf = valdf.merge(crsp1, on=['year', 'permno'])
val_crsp_mdf['vt'] = val_crsp_mdf['PV']*1000
val_crsp_mdf['be'] = val_crsp_mdf['be']*1000

In [15]:
val_df1 = val_crsp_mdf[['permno', 'ticker',  'exchcd', 'year', 'jdate', 'exchcd',
                        'vt','me', 'be', 
                        'logret_3', 'logret_6', 'logret_24', 'cumret_3', 'cumret_6', 'cumret_24',
                        'logret_12', 'logret_36', 'logret_48', 'logret_60', 
                        'cumret_12', 'cumret_36', 'cumret_48', 'cumret_60',]]
val_df1.sort_values(['permno','jdate'], ascending = [True,False], inplace=True)

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
val_df2 = val_df1.copy()
for m in [3, 6, 12, 24, 36, 48, 60]:
    val_df2[f'vt+{m}'] = val_df1.groupby('permno')['vt'].shift(m)
    val_df2[f'me+{m}'] = val_df1.groupby('permno')['me'].shift(m)
    val_df2[f'be+{m}'] = val_df1.groupby('permno')['be'].shift(m)

In [17]:
val_df3 = val_df2.copy()
for m in [3, 6, 12, 24, 36, 48, 60]:
    val_df3['vt'] = np.log(val_df2.vt)
    val_df3[f'vt+{m}'] = np.log(val_df2[f'vt+{m}'].astype(float))
    val_df3['me'] = np.log(val_df2['me'].astype(float))
    val_df3[f'me+{m}'] = np.log(val_df2[f'me+{m}'].astype(float))
    val_df3[f'r_t+{m}'] = val_df3[f'logret_{m}']
    val_df3[f'r*_t+{m}'] = val_df3[f'vt+{m}'] -  val_df3[f'vt']
    val_df3[f'r_unchg+{m}'] = - (1-0.975) * (val_df3['me'] -  val_df3['vt'])
    val_df3[f'r_chg+{m}'] = 0.975 * ( (val_df3[f'me+{m}'] -  val_df3[f'vt+{m}']) - ((val_df3[f'me'] -  val_df3['vt'])) )
    

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


In [18]:
val_df3 = val_df3[(val_df3.year >=1980) & (val_df3.year<=2020)]
val_df3 = val_df3.drop_duplicates(subset=['permno', 'jdate'])

In [19]:
val_df3['misp'] = val_df3['me'] - val_df3['vt']
val_df3 = val_df3.dropna(subset=['misp', 'r_t+60'])

In [20]:
val_df3[(val_df3.ticker=='AAPL') ]

,permno,ticker,exchcd,year,jdate,exchcd,vt,me,be,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_48,logret_60,cumret_12,cumret_36,cumret_48,cumret_60,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+48,me+48,be+48,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+48,r*_t+48,r_unchg+48,r_chg+48,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
203264,14593,AAPL,3,2019,2019-12-31,3,20.608597,20.989289,165551000.0,-0.430783,-0.430783,-0.430783,-0.350000,-0.350000,-0.350000,-0.430783,-0.430783,-0.430783,-0.430783,-0.350000,-0.350000,-0.350000,-0.350000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.430783,NaN,-0.009517,NaN,-0.430783,NaN,-0.009517,NaN,-0.430783,NaN,-0.009517,NaN,-0.430783,NaN,-0.009517,NaN,-0.430783,NaN,-0.009517,NaN,-0.430783,NaN,-0.009517,NaN,-0.430783,NaN,-0.009517,NaN,0.380692
203263,14593,AAPL,3,2019,2019-11-30,3,20.608597,20.895085,165551000.0,-0.336579,-0.336579,-0.336579,-0.285790,-0.285790,-0.285790,-0.336579,-0.336579,-0.336579,-0.336579,-0.285790,-0.285790,-0.285790,-0.285790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.336579,NaN,-0.007162,NaN,-0.336579,NaN,-0.007162,NaN,-0.336579,NaN,-0.007162,NaN,-0.336579,NaN,-0.007162,NaN,-0.336579,NaN,-0.007162,NaN,-0.336579,NaN,-0.007162,NaN,-0.336579,NaN,-0.007162,NaN,0.286488
203262,14593,AAPL,3,2019,2019-10-31,3,20.608597,20.823389,165551000.0,-0.262006,-0.262006,-0.262006,-0.230493,-0.230493,-0.230493,-0.262006,-0.262006,-0.262006,-0.262006,-0.230493,-0.230493,-0.230493,-0.230493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.262006,NaN,-0.005370,NaN,-0.262006,NaN,-0.005370,NaN,-0.262006,NaN,-0.005370,NaN,-0.262006,NaN,-0.005370,NaN,-0.262006,NaN,-0.005370,NaN,-0.262006,NaN,-0.005370,NaN,-0.262006,NaN,-0.005370,NaN,0.214792
203261,14593,AAPL,3,2019,2019-09-30,3,20.608597,20.718406,165551000.0,0.273754,-0.157029,-0.157029,0.314891,-0.145321,-0.145321,-0.157029,-0.157029,-0.157029,-0.157029,-0.145321,-0.145321,-0.145321,-0.145321,20.608597,20.989289,165551000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.273754,0.000000,-0.002745,0.264111,-0.157029,NaN,-0.002745,NaN,-0.157029,NaN,-0.002745,NaN,-0.157029,NaN,-0.002745,NaN,-0.157029,NaN,-0.002745,NaN,-0.157029,NaN,-0.002745,NaN,-0.157029,NaN,-0.002745,NaN,0.109809
203260,14593,AAPL,3,2019,2019-08-31,3,20.608597,20.664931,165551000.0,0.249972,-0.086607,-0.086607,0.283990,-0.082962,-0.082962,-0.086607,-0.086607,-0.086607,-0.086607,-0.082962,-0.082962,-0.082962,-0.082962,20.608597,20.895085,165551000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.249972,0.000000,-0.001408,0.224400,-0.086607,NaN,-0.001408,NaN,-0.086607,NaN,-0.001408,NaN,-0.086607,NaN,-0.001408,NaN,-0.086607,NaN,-0.001408,NaN,-0.086607,NaN,-0.001408,NaN,-0.086607,NaN,-0.001408,NaN,0.056334
203259,14593,AAPL,3,2019,2019-07-31,3,20.608597,20.685321,165551000.0,0.158691,-0.103315,-0.103315,0.171975,-0.098157,-0.098157,-0.103315,-0.103315,-0.103315,-0.103315,-0.098157,-0.098157,-0.098157,-0.098157,20.608597,20.823389,165551000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.158691,0.000000,-0.001918,0.134616,-0.103315,NaN,-0.001918,NaN,-0.103315,NaN,-0.001918,NaN,-0.103315,NaN,-0.001918,NaN,-0.103315,NaN,-0.001918,NaN,-0.103315,NaN,-0.001918,NaN,-0.103315,NaN,-0.001918,NaN,0.076724
203258,14593,AAPL,3,2019,2019-06-30,3,20.608597,20.614403,165551000.0,0.127331,0.401085,-0.029698,0.135793,0.493444,-0.029262,-0.029698,-0.029698,-0.029698,-0.029698,-0.029262,-0.029262,-0.029262,-0.029262,20.608597,20.718406,165551000.0,20.608597,20.989289,165551000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.127331,0.000000,-0.000145,0.101402,0.401085,0.

In [21]:
val_df3 = val_df3.replace([np.inf, -np.inf], np.nan)
val_df3.to_csv(f'{DATA_FOLDER}/decomp_dcf5y_PLS_1980_2019.csv')

In [22]:
val_df3.describe()

,permno,exchcd,year,exchcd,vt,me,be,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_48,logret_60,cumret_12,cumret_36,cumret_48,cumret_60,vt+3,me+3,be+3,vt+6,me+6,be+6,vt+12,me+12,be+12,vt+24,me+24,be+24,vt+36,me+36,be+36,vt+48,me+48,be+48,vt+60,me+60,be+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+48,r*_t+48,r_unchg+48,r_chg+48,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
count,1.251967e+06,1.251967e+06,1.251967e+06,1.251967e+06,1.251967e+06,1.251539e+06,1.136160e+06,1.251955e+06,1.251945e+06,1.251895e+06,1.251967e+06,1.251967e+06,1.251967e+06,1.251927e+06,1.251871e+06,1.251839e+06,1.251811e+06,1.251967e+06,1.251967e+06,1.251967e+06,1.251967e+06,1.187032e+06,1.222519e+06,1.110543e+06,1.123407e+06,1.194016e+06,1.085414e+06,1.000326e+06,1.138752e+06,1.036841e+06,865129.000000,1.038282e+06,9.476310e+05,767352.000000,947381.000000,8.656290e+05,691196.000000,864786.000000,7.905080e+05,632273.000000,789543.000000,7.219980e+05,1.251955e+06,1.187032e+06,1.251539e+06,1.186448e+06,1.251945e+06,1.123407e+06,1.251539e+06,1.122896e+06,1.251927e+06,1.000326e+06,1.251539e+06,999953.000000,1.251895e+06,865129.000000,1.251539e+06,864840.000000,1.251871e+06,767352.000000,1.251539e+06,767132.000000,1.251839e+06,691196.000000,1.251539e+06,690986.000000,1.251811e+06,632273.000000,1.251539e+06,632106.000000,1.251539e+06
mean,5.761467e+04,1.969155e+00,2.000554e+03,1.969155e+00,1.411035e+01,1.271118e+01,2.114990e+06,-3.187812e-03,-5.922707e-03,-1.644439e-02,2.670280e-02,5.371670e-02,2.150497e-01,-1.028540e-02,-1.659506e-02,-1.873005e-02,-1.874065e-02,1.092325e-01,3.217936e-01,4.234504e-01,5.344639e-01,1.415507e+01,1.273218e+01,2.143213e+06,1.420392e+01,1.275361e+01,2.172176e+06,1.431505e+01,1.279679e+01,2.231291e+06,14.408924,1.288358e+01,2.350586e+06,14.494930,12.980104,2.486091e+06,14.575497,13.075102,2.642837e+06,14.655968,13.172090,2.826100e+06,-3.187812e-03,1.850439e-02,3.499541e-02,-9.737134e-03,-5.922707e-03,3.895354e-02,3.499541e-02,-2.008939e-02,-1.028540e-02,8.638480e-02,3.499541e-02,-0.040527,-1.644439e-02,0.178959,3.499541e-02,-0.065056,-1.659506e-02,0.294925,3.499541e-02,-0.099715,-1.873005e-02,0.410670,3.499541e-02,-0.137857,-1.874065e-02,0.519025,3.499541e-02,-0.166237,-1.399817e+00
std,2.653820e+04,9.555592e-01,1.082818e+01,9.555592e-01,1.883035e+00,2.299931e+00,9.459749e+06,2.485671e-01,3.552902e-01,7.243236e-01,2.733774e-01,4.056570e-01,1.257388e+00,5.094961e-01,8.803044e-01,1.004243e+00,1.111623e+00,6.418774e-01,1.637649e+00,2.172317e+00,1.925319e+00,1.862343e+00,2.298041e+00,9.516291e+06,1.838224e+00,2.295785e+00,9.573283e+06,1.776480e+00,2.291424e+00,9.686699e+06,1.790714,2.284670e+00,9.878158e+06,1.793860,2.277883,1.006835e+07,1.786429,2.278702,1.041490e+07,1.788153,2.277867,1.087418e+07,2.485671e-01,4.549503e-01,5.136348e-02,5.088910e-01,3.552902e-01,6.576128e-01,5.136348e-02,7.301017e-01,5.094961e-01,9.731976e-01,5.136348e-02,1.058733,7.243236e-01,1.186312,5.136348e-02,1.305709,8.803044e-01,1.303433,5.136348e-02,1.443184,1.004243e+00,1.357022,5.136348e-02,1.505656,1.111623e+00,1.368797,5.136348e-02,1.527590,2.054539e+00
min,1.000100e+04,1.000000e+00,1.980000e+03,1.000000e+00,1.293335e+00,1.313388e+00,1.000000e+00,-9.478570e+00,-1.091677e+01,-1.484859e+01,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.231215e+01,-1.652222e+01,-1.874448e+01,-1.874448e+01,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,1.293335e+00,1.313388e+00,1.000000e+00,1.293335e+00,1.313388e+00,1.000000e+00,1.304352e+00,1.313388e+00,1.000000e+00,3.875636,1.313388e+00,1.000000e+00,3.875636,1.313388,1.000000e+00,3.875636,1.313388,1.000000e+00,4.554929,1.313388,7.000000e+00,-9.478570e+00,-1.460860e+01,-2.602439e-01,-1.117260e+01,-1.091677e+01,-1.460860e+01,-2.602439e-01,-1.105454e+01,-1.231215e+01,-1.460860e+01,-2.602439e-01,-11.186697,-1.484859e+01,-11.011787,-2.602439e-01,-10.806445,-1.652222e+01,